In [ ]:
!pip install scipy

In [ ]:
import scipy.io
import os
from PIL import Image
import numpy as np
import pandas as pd

In [ ]:
# Example directories
mat_dir = "../data/archive/dataset/data"
image_dir = "../data/mri_t1_dataset/images"
mask_dir = "../data/mri_t1_dataset/masks"

labels_map = {1: "meningioma", 2: "glioma", 3: "pituitary"}
metadata = []

for mat_file in os.listdir(mat_dir):
    if mat_file.endswith(".mat"):
        data = scipy.io.loadmat(os.path.join(mat_dir, mat_file))
        cjdata = data['cjdata']

        # Extract fields
        img = cjdata['image'][0,0]          # 2D array
        mask = cjdata['tumorMask'][0,0]     # 2D array
        label = int(cjdata['label'][0,0])
        pid = int(cjdata['PID'][0,0])

        label_name = labels_map[label]

        # Create directories if not exist
        os.makedirs(os.path.join(image_dir, label_name), exist_ok=True)
        os.makedirs(os.path.join(mask_dir, label_name), exist_ok=True)

        # Save image and mask as PNG
        filename = f"{mat_file.split('.')[0]}.png"
        Image.fromarray(img).save(os.path.join(image_dir, label_name, filename))
        Image.fromarray((mask*255).astype(np.uint8)).save(os.path.join(mask_dir, label_name, filename))

        # Add to metadata
        metadata.append([filename, label_name, pid])

# Save metadata
df = pd.DataFrame(metadata, columns=["filename", "label", "PID"])
df.to_csv("brain_tumor_dataset/metadata.csv", index=False)